# Entraînement d'un modèle d'embeddings sur toutes les données

# Import des différents packages

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras

2024-01-15 21:00:49.869341: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 21:00:49.876782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 21:00:49.990997: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 21:00:49.991080: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 21:00:49.991145: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

## Import des données

In [14]:


# Model import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Load your dataset (assuming you have a CSV file)

df_adverse = pd.read_csv("Data/adverse_reviews.csv")
df_adverse = df_adverse[df_adverse["rate"] != -1]


df_climate = pd.read_csv("Data/climate_reviews.csv")
df_climate = df_climate[df_climate["rate"] != -1]




df_governance = pd.read_csv("Data/governance_reviews.csv")
df_governance = df_governance[df_governance["rate"] != -1]



df_organic = pd.read_csv("Data/organic_reviews.csv")
df_organic = df_organic[df_organic["rate"] != -1]

df_social = pd.read_csv("Data/social_reviews.csv")
df_social = df_social[df_social["rate"] != -1]


df_waste = pd.read_csv("Data/waste_reviews.csv")
df_waste = df_waste[df_waste["rate"] != -1]

df = pd.read_csv("GptData/everything.csv")

df_all = pd.concat([df_adverse,df_climate,df_governance,df_organic,df_social,df_waste],axis=0,ignore_index=True)


# Change the colone name

df_all['label'] = df_all["rate"]
df_all = df_all.drop("rate",axis=1)

df_all = pd.concat([df, df_all])
print(df_all)



                                                 review  label
0     A total disappointment. Their 'ecological appr...      0
1     An ecological pretension devoid of meaning. Th...      0
2     Disappointing. Their alleged ecological consci...      0
3     A restaurant that boasts of being ecological b...      0
4     Ecological facades. Their discourse on ecology...      0
...                                                 ...    ...
1145  While not achieving top-tier performance in wa...      3
1146  As someone who dined at this venue, the waste ...      3
1147  In my experience at this joint, the waste mana...      3
1148  First time eating here and it lived up to all ...      4
1149  I absolutely love this chain. The salads are a...      2

[2054 rows x 2 columns]


## Séparation des données d'entraînement, de test et de validation.

In [16]:

train_data, test_data = train_test_split(df_all, test_size=0.2, random_state=42)
test_data , val_data = train_test_split(test_data, test_size=0.2, random_state=42)

## Tokénisation des données

In [15]:


# Tokenize and pad the text data
max_words = 10000  # Choose the maximum number of words in your vocabulary
max_len = 100  # Choose the maximum length of your sequences

# Tokenizer init 
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['review'])


# Encode the labels
# we can take the data that have already been tokenized


# Applying the tokenizer to the reviews with a padding
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=max_len)
print(X_train.shape)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=max_len)
X_val = pad_sequences(tokenizer.texts_to_sequences(val_data['review']), maxlen=max_len)

(1643, 100)


## Encodage des labels

In [18]:
# Encode the labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['label'])
y_test = label_encoder.transform(test_data['label'])
y_val = label_encoder.transform(val_data['label'])




## Construction du modèle

In [23]:



model_grade = Sequential()
model_grade.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model_grade.add(LSTM(128))
model_grade.add(Dropout(0.3))
model_grade.add(Dense(64, activation='relu'))
model_grade.add(Dropout(0.1))
model_grade.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_grade.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model_grade.summary()




Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 64)           640000    
                                                                 
 lstm_3 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 6)                 390       
                                                                 
Total params: 747462 (2.85 MB)
Trainable params: 74746

## Entraînement du modèle

In [24]:
model_grade.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
52/52 [==============================] - 5s 70ms/step - loss: 1.7385 - accuracy: 0.2818 - val_loss: 1.4651 - val_accuracy: 0.3415
Epoch 2/10
52/52 [==============================] - 4s 68ms/step - loss: 1.1723 - accuracy: 0.5265 - val_loss: 0.9826 - val_accuracy: 0.6951
Epoch 3/10
52/52 [==============================] - 4s 71ms/step - loss: 0.6311 - accuracy: 0.8089 - val_loss: 0.5743 - val_accuracy: 0.8140
Epoch 4/10
52/52 [==============================] - 3s 66ms/step - loss: 0.2852 - accuracy: 0.9044 - val_loss: 0.5275 - val_accuracy: 0.8232
Epoch 5/10
52/52 [==============================] - 3s 64ms/step - loss: 0.1977 - accuracy: 0.9355 - val_loss: 0.4731 - val_accuracy: 0.8262
Epoch 6/10
52/52 [==============================] - 3s 66ms/step - loss: 0.1362 - accuracy: 0.9550 - val_loss: 0.5802 - val_accuracy: 0.8293
Epoch 7/10
52/52 [==============================] - 3s 66ms/step - loss: 0.1504 - accuracy: 0.9446 - val_loss: 0.4522 - val_accuracy: 0.8720
Epoch 8/10
52

## Résultats

In [25]:
loss , acc = model_grade.evaluate(X_val,y_val,verbose=2)
print("Accuracy new data",acc)


new_texts = ["Best managed McDonald's I've ever seen!,","Yo, socially, this joint is kinda in the middle. Moves and inclusivity are cool, like hosting a regular hangout. A chill experience, nothing too wild.","A total disappointment. Their 'ecological approach' was a facade, the omnipresent plastic being blatant proof.","Discovering the treasures within these walls reveals a masterpiece that surpasses expectations. The chef, a virtuoso in directing the kitchen, conducts an orchestra of flavors with finesse. It's not merely dining; it's an experience that transcends the realms of artistry."]
new_sequences = pad_sequences(tokenizer.texts_to_sequences(new_texts), maxlen=max_len)

predictions = model_grade.predict(new_sequences)
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))

print(predicted_labels)

3/3 - 0s - loss: 0.4784 - accuracy: 0.9036 - 81ms/epoch - 27ms/step
Accuracy new data 0.9036144614219666
1/1 [==============================] - 0s 273ms/step
[4 2 0 5]
